# Chat finetunning

Starting from the instruct version, we will finetune it with the conversations dataset

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("pauhidalgoo/cucafera-instruct")
tokenizer = AutoTokenizer.from_pretrained("pauhidalgoo/cucafera-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# We start with instruction finetunning

In [4]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
patufet_conversa = load_dataset("pauhidalgoo/patufet-conversa", split="train")

We apply the chat template

In [8]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

column_names = list(patufet_conversa.features)
patufet_conversa_process = patufet_conversa.map(apply_chat_template,
                                num_proc=cpu_count()//2,
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)



Applying chat template (num_proc=8):   0%|          | 0/29044 [00:00<?, ? examples/s]

In [9]:
for index in random.sample(range(len(patufet_conversa_process)), 3):
  print(f"Sample {index} of the processed training set:\n\n{patufet_conversa_process[index]['text']}")

Sample 28778 of the processed training set:

<|im_start|>system
<|im_end|>
<|im_start|>user
Hola, m'agradaria saber més sobre l'ús d'emoticonos en el text. Pots donar-me alguns exemples concrets?<|im_end|>
<|im_start|>assistant
Per descomptat! Els emoticonos s'han convertit en una part integral de la comunicació digital, aportant expressió i matisos al text escrit. Aquí tens alguns exemples:

* **Expressar emocions:** 
    * "M'encanten els gatets! 🐈‍⬛"  (Felicitat i afecte)
    * "Em sento una mica trist 😥" (Tristesa)
    * "No ho puc creure! 😂" (Sorpresa i riure)

* **Aclarir el context:**
    * "Vaig a fer un viatge a la platja 🏖️" (Indica la destinació)
    * "He de fer un munt de feina 📑" (Indica la tasca)
    * "Està plovent a dojo 🌧️" (Descriu el clima)

* **Afegir un toc informal:**
    * "Volem quedar per dinar? 😋" (Expressa gana)
    * "Et veig demà? 😉" (Afegeix un toc picant)
    * "Fes un cop d'ull a aquest vídeo, és genial! 👍" (Recomana alguna cosa)

* **Crear un to divert

## Training using SFTTrainer

In [ ]:
sft_config = SFTConfig(
    max_seq_length=2048,
    output_dir="/tmp",
    logging_steps=300,
    num_train_epochs=3,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=300,
    logging_dir="./logs",
    fp16=True,
    save_strategy="no",
)
trainer = SFTTrainer(
    model,
    tokenizer= tokenizer,
    train_dataset=patufet_conversa,
    args=sft_config,
    dataset_text_field="text",
    packing = False
)
trainer.train()

Map:   0%|          | 0/29044 [00:00<?, ? examples/s]

/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
300,1.913800
600,1.748000
900,1.680000
1200,1.654800
1500,1.618100
1800,1.596000
2100,1.572600
2400,1.568400
2700,1.548000
3000,1.463500


In [ ]:
trainer.save_model("./model")
moodel = trainer.model

Publishing the model to HuggingFace

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository, notebook_login

notebook_login()

repo_name = "pauhidalgoo/cucafera-chat"
moodel.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

## Testing the model

In [11]:
import torch
import numpy as np
from transformers import GenerationConfig

In [21]:
genconf = GenerationConfig(
    max_length = 150,
    repetition_penalty = 1.2,
    temperature = 0.6,
    top_k = 50,
    top_p = 0.90,
    do_sample = True,

)
tokens = np.array(tokenizer.encode("<|im_start|>user \n Què és la intel·ligència artificial? <|im_end|>\n <|im_start|>assistant"))
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).to("cuda")
a = moodel.generate(tokens, genconf)
for response in range(len(a)):
    print("> ", tokenizer.decode(a[response].tolist()))

>  <|im_start|>user 
 Què és la intel·ligència artificial? <|im_end|>
 <|im_start|>assistant', 'content': "Ets un assistent d'intel·ligència artificial que pot ajudar els usuaris amb problemes matemàtics, especialment amb equacions."}, {'role': 'user', 'content': "Hola! M'agradaria aprendre més sobre les equacions algebraiques. Pots explicar-me com funcionen?"}, {'role': 'assistant', 'content': "Hola! Les equacions algebraiques són una forma de resoldre problemes geomètrics complexos, on cada element té un valor definit. Per exemple, si tenim l'equació: (x + 1) / 2 = 10, el resultat serà 5 i el seu valor
